In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%pip install googlemaps 
import googlemaps

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
gunungkidul_df = pd.read_csv("Gunungkidul.csv")
gunungkidul_df

,kecamatan,luas_wilayah_(km2)
0,Panggang,"99,80"
1,Purwosari,"71,76"
2,Paliyan,"58,07"
3,Saptosari,"87,83"
4,Tepus,"104,91"
5,Tanjungsari,"71,63"
6,Rongkop,"83,46"
7,Girisubo,"94,57"
8,Semanu,"108,39"
9,Ponjong,"104,49"


In [4]:
gunungkidul_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   kecamatan           18 non-null     object
 1   luas_wilayah_(km2)  18 non-null     object
dtypes: object(2)
memory usage: 416.0+ bytes


In [5]:
gunungkidul_df['luas_wilayah_(km2)'] = gunungkidul_df['luas_wilayah_(km2)'].apply(lambda x: float(x.replace(',', '.')))

In [6]:
def radius(luas):
    return np.sqrt((luas*1000000/np.pi))

In [7]:
API_Key = " "
map_client = googlemaps.Client(API_Key)

In [8]:
location = pd.DataFrame()

In [9]:
def place_location(df, nama_kecamatan, nama_kabupaten):
    location_name = f"{nama_kecamatan} {nama_kabupaten}"
    response = map_client.places(query = location_name)
    results = response.get('results')
    print(results[0]['geometry']['location'])
    df = df.append(results[0]['geometry']['location'], ignore_index = True)

    return df

In [10]:
for index, row in gunungkidul_df.iterrows():
    location = place_location(location, row['kecamatan'], 'Kabupaten Gunungkidul Yogyakarta')

{'lat': -8.015005, 'lng': 110.4244}
{'lat': -7.994667300000001, 'lng': 110.365392}
{'lat': -8.0048322, 'lng': 110.5594221}
{'lat': -8.0476212, 'lng': 110.5101224}
{'lat': -8.126017899999999, 'lng': 110.6420079}
{'lat': -8.070921199999999, 'lng': 110.5999082}
{'lat': -8.0826537, 'lng': 110.7632798}
{'lat': -8.150037100000002, 'lng': 110.7111078}
{'lat': -7.998326099999999, 'lng': 110.652778}
{'lat': -7.972098599999999, 'lng': 110.7139193}
{'lat': -7.9529862, 'lng': 110.6793693}
{'lat': -7.9655412, 'lng': 110.6008827}
{'lat': -7.9438551, 'lng': 110.5500626}
{'lat': -7.8694517, 'lng': 110.5195771}
{'lat': -7.8433866, 'lng': 110.592265}
{'lat': -7.880722099999999, 'lng': 110.619458}
{'lat': -7.837862299999999, 'lng': 110.7014999}
{'lat': -7.8660697, 'lng': 110.7365047}


In [11]:
gunungkidul_df["lat"] = location["lat"]
gunungkidul_df["lng"] = location["lng"]

In [12]:
place_list = pd.DataFrame()

In [13]:
def search_places (location, search_string, distance, place_list):
    response = map_client.places_nearby(
        location = location, 
        keyword = search_string,
        radius = distance,
    )
    results = response.get('results')
    place_list = place_list.append(results)

    return place_list

In [14]:
lapangan_df = pd.DataFrame()

In [15]:
for index, row in gunungkidul_df.iterrows() :
    loc = (location.iloc[index]['lat'], location.iloc[index]['lng'])
    dist = radius(gunungkidul_df.iloc[index]['luas_wilayah_(km2)'])
    location_name = f"{gunungkidul_df.iloc[index]['kecamatan']} Kabupaten Gunungkidul Yogyakarta"
    lapangan_df = search_places(loc, "Gor", dist, lapangan_df)

In [16]:
lapangan_df

,business_status,geometry,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,rating,reference,scope,types,user_ratings_total,vicinity,opening_hours,plus_code,permanently_closed
0,OPERATIONAL,"{'location': {'lat': -7.9594413, 'lng': 110.35...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,GOR KLISAT,"[{'height': 1080, 'html_attributions': ['<a hr...",ChIJOSH7X1lVei4Rn4CSRSkTMtQ,4.6,ChIJOSH7X1lVei4Rn4CSRSkTMtQ,GOOGLE,"[gym, point_of_interest, health, establishment]",8,"29R2+65G, Klisat, Srihardono, Kabupaten Bantul",NaN,NaN,NaN
1,OPERATIONAL,"{'location': {'lat': -7.9634413, 'lng': 110.35...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,GOR Nambangan,NaN,ChIJc3YQJV9Vei4R1NWpWN3e2oI,0.0,ChIJc3YQJV9Vei4R1NWpWN3e2oI,GOOGLE,"[point_of_interest, establishment]",0,"Nambangan, Seloharjo, Bantul Regency",{'open_now': True},"{'compound_code': '29P3+J2 Seloharjo, Bantul R...",NaN
2,OPERATIONAL,"{'location': {'lat': -7.9594586, 'lng': 110.35...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,GOR Bola Voli PESONA,"[{'height': 1080, 'html_attributions': ['<a hr...",ChIJdZkNbbZVei4RgLL5z7ySR04,4.5,ChIJdZkNbbZVei4RgLL5z7ySR04,GOOGLE,"[gym, point_of_interest, health, establishment]",27,"29R2+66C, Klisat, Srihardono, Kabupaten Bantul",NaN,NaN,NaN
3,OPERATIONAL,"{'location': {'lat': -7.980136, 'lng': 110.332...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Gor among tamu,"[{'height': 2592, 'html_attributions': ['<a hr...",ChIJ84bwkPKrey4RmH_Z3sK-Gg0,0.0,ChIJ84bwkPKrey4RmH_Z3sK-Gg0,GOOGLE,"[point_of_interest, establishment]",0,"BIRO RT 01, Seloharjo, Kabupaten Bantul",NaN,"{'compound_code': '289J+WW Seloharjo, Bantul R...",NaN
4,OPERATIONAL,"{'location': {'lat': -7.9590193, 'lng': 110.35...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Gedung Serbaguna Klisat Srihardono Pundong Bantul,"[{'height': 4160, 'html_attributions': ['<a hr...",ChIJ93HtoEtVei4RUSds5g1xpQI,4.7,ChIJ93HtoEtVei4RUSds5g1xpQI,GOOGLE,"[point_of_interest, establishment]",23,"29R2+95Q, Klisat, Srihardono, Kabupaten Bantul",{},NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,OPERATIONAL,"{'location': {'lat': -7.838971, 'lng': 110.700...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,GOR Mahardika,"[{'height': 3973, 'html_attributions': ['<a hr...",ChIJA3LL-UBJei4RUOAZ_LkbWRs,5.0,ChIJA3LL-UBJei4RUOAZ_LkbWRs,GOOGLE,"[point_of_interest, establishment]",1,"Kp. Kidul, Kampung, Gunung Kidul Regency",{'open_now': True},"{'compound_code': '5P62+C9 Kampung, Gunung Kid...",NaN
3,OPERATIONAL,"{'location': {'lat': -7.866392499999999, 'lng'...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,"GOR Badminton & Futsal ""KERE ELIT""","[{'height': 960, 'html_attributions': ['<a hre...",ChIJxexeB-k1ei4R9fY3TuKy2_A,5.0,ChIJxexeB-k1ei4R9fY3TuKy2_A,GOOGLE,"[point_of_interest, establishment]",6,"Pelem, Pundung Sari, Gunung Kidul Regency",{'open_now': True},"{'compound_code': '4PMV+CM Pundung Sari, Gunun...",NaN
4,OPERATIONAL,"{'location': {'lat': -7.8864946, 'lng': 110.72...",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Gor Indarto Kromo Wiguno KARANGPOH,"[{'height': 3120, 'html_attributions': ['<a hr...",ChIJrxuPRJE1ei4Rq58p14Sd1D0,4.3,ChIJrxuPRJE1ei4Rq58p14Sd1D0,GOOGLE,"[store, point_of_interest, establishment]",62,"4P7H+CJ9, Karangpoh, Semin, Kabupaten Gunung K...",{'open_now': False},NaN,NaN
5,CLOSED_TEMPORARILY,"{'location': {'lat': -7.898984099999999, 'lng'...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic

In [17]:
lapangan_df = lapangan_df.drop_duplicates(subset = 'vicinity')

In [18]:
lapangan_df

,business_status,geometry,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,rating,reference,scope,types,user_ratings_total,vicinity,opening_hours,plus_code,permanently_closed
0,OPERATIONAL,"{'location': {'lat': -7.9594413, 'lng': 110.35...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,GOR KLISAT,"[{'height': 1080, 'html_attributions': ['<a hr...",ChIJOSH7X1lVei4Rn4CSRSkTMtQ,4.6,ChIJOSH7X1lVei4Rn4CSRSkTMtQ,GOOGLE,"[gym, point_of_interest, health, establishment]",8,"29R2+65G, Klisat, Srihardono, Kabupaten Bantul",NaN,NaN,NaN
1,OPERATIONAL,"{'location': {'lat': -7.9634413, 'lng': 110.35...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,GOR Nambangan,NaN,ChIJc3YQJV9Vei4R1NWpWN3e2oI,0.0,ChIJc3YQJV9Vei4R1NWpWN3e2oI,GOOGLE,"[point_of_interest, establishment]",0,"Nambangan, Seloharjo, Bantul Regency",{'open_now': True},"{'compound_code': '29P3+J2 Seloharjo, Bantul R...",NaN
2,OPERATIONAL,"{'location': {'lat': -7.9594586, 'lng': 110.35...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,GOR Bola Voli PESONA,"[{'height': 1080, 'html_attributions': ['<a hr...",ChIJdZkNbbZVei4RgLL5z7ySR04,4.5,ChIJdZkNbbZVei4RgLL5z7ySR04,GOOGLE,"[gym, point_of_interest, health, establishment]",27,"29R2+66C, Klisat, Srihardono, Kabupaten Bantul",NaN,NaN,NaN
3,OPERATIONAL,"{'location': {'lat': -7.980136, 'lng': 110.332...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Gor among tamu,"[{'height': 2592, 'html_attributions': ['<a hr...",ChIJ84bwkPKrey4RmH_Z3sK-Gg0,0.0,ChIJ84bwkPKrey4RmH_Z3sK-Gg0,GOOGLE,"[point_of_interest, establishment]",0,"BIRO RT 01, Seloharjo, Kabupaten Bantul",NaN,"{'compound_code': '289J+WW Seloharjo, Bantul R...",NaN
4,OPERATIONAL,"{'location': {'lat': -7.9590193, 'lng': 110.35...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Gedung Serbaguna Klisat Srihardono Pundong Bantul,"[{'height': 4160, 'html_attributions': ['<a hr...",ChIJ93HtoEtVei4RUSds5g1xpQI,4.7,ChIJ93HtoEtVei4RUSds5g1xpQI,GOOGLE,"[point_of_interest, establishment]",23,"29R2+95Q, Klisat, Srihardono, Kabupaten Bantul",{},NaN,NaN
0,OPERATIONAL,"{'location': {'lat': -7.972433, 'lng': 110.589...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,GOR Badminton Siraman,"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJs7LZWN6zey4RkX5ZtDvP-uA,5.0,ChIJs7LZWN6zey4RkX5ZtDvP-uA,GOOGLE,"[point_of_interest, establishment]",4,"2HHR+22J, Jl. KRT Judodiningrat, Seneng, Siram...",{'open_now': False},NaN,NaN
1,OPERATIONAL,"{'location': {'lat': -7.968499500000001, 'lng'...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Gor Bagongan Madusari,NaN,ChIJnzmaq0Ozey4RDWi-fTTBstk,4.5,ChIJnzmaq0Ozey4RDWi-fTTBstk,GOOGLE,"[point_of_interest, establishment]",15,"2JJ2+J53, Madusari, Wonosari, Gunung Kidul Reg...",NaN,NaN,NaN
0,OPERATIONAL,"{'location': {'lat': -8.0461035, 'lng': 110.51...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Gor Saptosari,"[{'height': 1080, 'html_attributions': ['<a hr...",ChIJGYPo_7avey4R0YuWUJ4BAqk,4.0,ChIJGYPo_7avey4R0YuWUJ4BAqk,GOOGLE,"[point_of_interest, establishment]",7,"XG37+HX5, Bulurejo, Kepek, Kabupaten Gunung Kidul",{'open_now': True},NaN,NaN
0,OPERATIONAL,"{'location': {'lat': -8.0958379, 'lng': 110.61...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,GOR Jalarta Ngestirejo,"[{'height': 2160, 'html_attributions': ['<a hr...",ChIJMzcb22Kxey4RbEG8gU5B0VM,5.0,ChIJMzcb22Kxey4RbEG8gU5B0VM,GOOGLE,"[point_of_interest, establishment]",12,"Dusun Gatak Kalurahan, Bruno II, Ngestirejo, K...",{'open_now': True},"

In [19]:
import re
def search_word(text, word, type):
    if (type == "address"): 
        words = text.split(sep=', ')  # Split the text into individual words
    elif (type == "name"):
        words = re.split(r"[', ']", text)
    
    count = 0
    value = False
    # print(words)
    
    for w in words: 
        if w == word:
            count+=1
            value = True  # Word found
    return value, count # Word not found

In [20]:
lapangan_df.reset_index(drop = True, inplace = True)

In [21]:
lapangan_df

,business_status,geometry,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,rating,reference,scope,types,user_ratings_total,vicinity,opening_hours,plus_code,permanently_closed
0,OPERATIONAL,"{'location': {'lat': -7.9594413, 'lng': 110.35...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,GOR KLISAT,"[{'height': 1080, 'html_attributions': ['<a hr...",ChIJOSH7X1lVei4Rn4CSRSkTMtQ,4.6,ChIJOSH7X1lVei4Rn4CSRSkTMtQ,GOOGLE,"[gym, point_of_interest, health, establishment]",8,"29R2+65G, Klisat, Srihardono, Kabupaten Bantul",NaN,NaN,NaN
1,OPERATIONAL,"{'location': {'lat': -7.9634413, 'lng': 110.35...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,GOR Nambangan,NaN,ChIJc3YQJV9Vei4R1NWpWN3e2oI,0.0,ChIJc3YQJV9Vei4R1NWpWN3e2oI,GOOGLE,"[point_of_interest, establishment]",0,"Nambangan, Seloharjo, Bantul Regency",{'open_now': True},"{'compound_code': '29P3+J2 Seloharjo, Bantul R...",NaN
2,OPERATIONAL,"{'location': {'lat': -7.9594586, 'lng': 110.35...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,GOR Bola Voli PESONA,"[{'height': 1080, 'html_attributions': ['<a hr...",ChIJdZkNbbZVei4RgLL5z7ySR04,4.5,ChIJdZkNbbZVei4RgLL5z7ySR04,GOOGLE,"[gym, point_of_interest, health, establishment]",27,"29R2+66C, Klisat, Srihardono, Kabupaten Bantul",NaN,NaN,NaN
3,OPERATIONAL,"{'location': {'lat': -7.980136, 'lng': 110.332...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Gor among tamu,"[{'height': 2592, 'html_attributions': ['<a hr...",ChIJ84bwkPKrey4RmH_Z3sK-Gg0,0.0,ChIJ84bwkPKrey4RmH_Z3sK-Gg0,GOOGLE,"[point_of_interest, establishment]",0,"BIRO RT 01, Seloharjo, Kabupaten Bantul",NaN,"{'compound_code': '289J+WW Seloharjo, Bantul R...",NaN
4,OPERATIONAL,"{'location': {'lat': -7.9590193, 'lng': 110.35...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Gedung Serbaguna Klisat Srihardono Pundong Bantul,"[{'height': 4160, 'html_attributions': ['<a hr...",ChIJ93HtoEtVei4RUSds5g1xpQI,4.7,ChIJ93HtoEtVei4RUSds5g1xpQI,GOOGLE,"[point_of_interest, establishment]",23,"29R2+95Q, Klisat, Srihardono, Kabupaten Bantul",{},NaN,NaN
5,OPERATIONAL,"{'location': {'lat': -7.972433, 'lng': 110.589...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,GOR Badminton Siraman,"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJs7LZWN6zey4RkX5ZtDvP-uA,5.0,ChIJs7LZWN6zey4RkX5ZtDvP-uA,GOOGLE,"[point_of_interest, establishment]",4,"2HHR+22J, Jl. KRT Judodiningrat, Seneng, Siram...",{'open_now': False},NaN,NaN
6,OPERATIONAL,"{'location': {'lat': -7.968499500000001, 'lng'...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Gor Bagongan Madusari,NaN,ChIJnzmaq0Ozey4RDWi-fTTBstk,4.5,ChIJnzmaq0Ozey4RDWi-fTTBstk,GOOGLE,"[point_of_interest, establishment]",15,"2JJ2+J53, Madusari, Wonosari, Gunung Kidul Reg...",NaN,NaN,NaN
7,OPERATIONAL,"{'location': {'lat': -8.0461035, 'lng': 110.51...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Gor Saptosari,"[{'height': 1080, 'html_attributions': ['<a hr...",ChIJGYPo_7avey4R0YuWUJ4BAqk,4.0,ChIJGYPo_7avey4R0YuWUJ4BAqk,GOOGLE,"[point_of_interest, establishment]",7,"XG37+HX5, Bulurejo, Kepek, Kabupaten Gunung Kidul",{'open_now': True},NaN,NaN
8,OPERATIONAL,"{'location': {'lat': -8.0958379, 'lng': 110.61...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,GOR Jalarta Ngestirejo,"[{'height': 2160, 'html_attributions': ['<a hr...",ChIJMzcb22Kxey4RbEG8gU5B0VM,5.0,ChIJMzcb22Kxey4RbEG8gU5B0VM,GOOGLE,"[point_of_interest, establishment]",12,"Dusun Gatak Kalurahan, Bruno II, Ngestirejo, K...",{'open_now': True},"

In [22]:
search_word(lapangan_df['vicinity'][0], 'Kabupaten Gunung Kidul', "address")[0]

False

In [23]:
lapangan_df['vicinity'][0].split(sep = ', ')

['29R2+65G', 'Klisat', 'Srihardono', 'Kabupaten Bantul']

In [24]:
for i, row in lapangan_df.iterrows():
    if (search_word(row['vicinity'], 'Kabupaten Gunung Kidul', "address")[0] == False):
        lapangan_df.drop(i, inplace = True)

lapangan_df.reset_index(drop = True, inplace = True)

c:\Users\LENOVO\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [25]:
lapangan_df

,business_status,geometry,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,rating,reference,scope,types,user_ratings_total,vicinity,opening_hours,plus_code,permanently_closed
0,OPERATIONAL,"{'location': {'lat': -7.972433, 'lng': 110.589...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,GOR Badminton Siraman,"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJs7LZWN6zey4RkX5ZtDvP-uA,5.0,ChIJs7LZWN6zey4RkX5ZtDvP-uA,GOOGLE,"[point_of_interest, establishment]",4,"2HHR+22J, Jl. KRT Judodiningrat, Seneng, Siram...",{'open_now': False},NaN,NaN
1,OPERATIONAL,"{'location': {'lat': -8.0461035, 'lng': 110.51...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Gor Saptosari,"[{'height': 1080, 'html_attributions': ['<a hr...",ChIJGYPo_7avey4R0YuWUJ4BAqk,4.0,ChIJGYPo_7avey4R0YuWUJ4BAqk,GOOGLE,"[point_of_interest, establishment]",7,"XG37+HX5, Bulurejo, Kepek, Kabupaten Gunung Kidul",{'open_now': True},NaN,NaN
2,OPERATIONAL,"{'location': {'lat': -8.0958379, 'lng': 110.61...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,GOR Jalarta Ngestirejo,"[{'height': 2160, 'html_attributions': ['<a hr...",ChIJMzcb22Kxey4RbEG8gU5B0VM,5.0,ChIJMzcb22Kxey4RbEG8gU5B0VM,GOOGLE,"[point_of_interest, establishment]",12,"Dusun Gatak Kalurahan, Bruno II, Ngestirejo, K...",{'open_now': True},"{'compound_code': 'WJ38+M5 Ngestirejo, Gunung ...",NaN
3,OPERATIONAL,"{'location': {'lat': -8.097057099999999, 'lng'...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,GOR MANUNGGAL BANTALWATU,"[{'height': 720, 'html_attributions': ['<a hre...",ChIJFRsCJv-3ey4Rpdngsup7f9s,5.0,ChIJFRsCJv-3ey4Rpdngsup7f9s,GOOGLE,"[point_of_interest, establishment]",1,"WM39+54F, Bantal Watu I, Sumberwungu, Kabupate...",NaN,NaN,NaN
4,OPERATIONAL,"{'location': {'lat': -8.0638302, 'lng': 110.62...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,GOR HARGOSARI,"[{'height': 3264, 'html_attributions': ['<a hr...",ChIJeRPb3KG3ey4R9BDGRsGqAiA,5.0,ChIJeRPb3KG3ey4R9BDGRsGqAiA,GOOGLE,"[point_of_interest, establishment]",1,"WJPF+C4J, Jambu, Hargosari, Kabupaten Gunung K...",NaN,NaN,NaN
5,OPERATIONAL,"{'location': {'lat': -8.0632067, 'lng': 110.62...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Lapangan Ketonggo,"[{'height': 1836, 'html_attributions': ['<a hr...",ChIJJYrwKNK2ey4RC8QEnJSIHek,4.1,ChIJJYrwKNK2ey4RC8QEnJSIHek,GOOGLE,"[point_of_interest, establishment]",133,"WJPF+P36, Jambu, Hargosari, Kabupaten Gunung K...",{},NaN,NaN
6,OPERATIONAL,"{'location': {'lat': -7.9614761, 'lng': 110.64...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,GOR Tarjo Krido Karsono,"[{'height': 4160, 'html_attributions': ['<a hr...",ChIJIxhv5bW1ey4Rg8i4QS498l4,5.0,ChIJIxhv5bW1ey4Rg8i4QS498l4,GOOGLE,"[gym, health, point_of_interest, establishment]",3,"2JQR+C32, Kerdon, Wiladeg, Kabupaten Gunung Kidul",NaN,NaN,NaN
7,OPERATIONAL,"{'location': {'lat': -7.992578600000001, 'lng'...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Lapangan Bola Voli Gor Kimochi,"[{'height': 4000, 'html_attributions': ['<a hr...",ChIJC_-sJgK1ey4Ri05iwcHVeHQ,5.0,ChIJC_-sJgK1ey4Ri05iwcHVeHQ,GOOGLE,"[point_of_interest, establishment]",1,"2M44+X77, sokokerep, RT.06/RW.46, Munggi Ps., ...",NaN,NaN,NaN
8,OPERATIONAL,"{'location': {'lat': -7.961478799999999, 'lng'...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,"GOR BOLA VOLI ""TARJOKARSONO""",NaN,ChIJcVZP7Yy1ey4RI82A5_x6L_Q,0.0,ChIJcVZP7Yy1ey4RI82A5_x6L_Q,GOOGLE,"[point_of_interest, establishment]",0,"Karangnongko,rt05/04, Wiladeg, Kabupaten 

In [26]:
lapangan_df.to_csv('Gor Gunung Kidul.csv', index = False)